In [3]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [12]:
# Importing required packages for this homework
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
#from IPython.display import HTML, display
#import tabulate
%matplotlib inline

def rounds(item):
    ret = float('%.1f'% (item * 10/10))
    return(ret)

path = os.getcwd()
print(path)

C:\Users\butle175\Downloads\AAE537-master


In [4]:
# Atmospheric model created using the NASA equation found at:
# https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html
class atmosphere:
    def __init__(self, val, valGiven = 0,units = "SI"):
        #Convert from US to SI
        if units != "SI" and valGiven == 0:
            val = val / 3.281
        if units != "SI" and valGiven == 1:
            val = val * 0.04788
        #0 implies the given value is an altitude
        #1 implies the given value is a pressure
        if valGiven == 0:
            self.h = val
        elif valGiven == 1:
            self.P = val
        else:
            print("Not a valid 'valGiven' parameter.")
    def hCalc(self):
        if self.h < 11000:
            self.T = 15.04 - 0.00649*self.h #K
            self.P = 101.29 * ((self.T + 273.1)/288.08)**(5.256) #kPa
        elif self.h < 25000:
            self.T = -56.46
            self.P = 22.65 * np.exp(1.73 - 0.000157*self.h)
        elif self.h > 24999:
            self.T = -131.21 + 0.00299*self.h
            self.P = 2.488 * ((self.T + 273.1)/216.6)**(-11.388)
        self.rho = self.P / (0.2869 * (self.T + 273.1))
    def PCalc(self):
        if self.P > 22.632:
            self.T = (288.08*(self.P/101.29)**(1/5.256))-273.1
            self.h = (self.T - 15.04)/(-0.00649)
        elif self.P > 0.1113586:
            self.T = -56.46
            self.h = (1.73 - np.log((self.P/22.65)))/(0.000157)
        else:
            self.T = (216.6*(self.P/2.488)**(1/(-11.388)))-273.1
            self.h = (self.T + 131.21)/0.00299
        self.rho = self.P / (0.2869 * (self.T + 273.1))
# ISENTROPIC RELATIONS
def totPres(Ps, M,gamma=1.4):
    total = Ps * ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(total)
def statPres(Pt, M, gamma=1.4):
    static = Pt / ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(static)
def totTemp(Ts, M, gamma=1.4):
    total = Ts * ((1+((gamma-1)/2)*M**2))
    return(total)
def statTemp(Tt, M, gamma=1.4):
    static = Tt / ((1+((gamma-1)/2)*M**2))
    return(static) 

In [14]:
#Getting the volume of the fuselodge
#Vol = 1/3 pi (r1^2 + r1r2 + r2^2)h
r1 = 24.59 * 0.0254 #meters
r2 = 42.14 * 0.0254 #meters
h = 379.73 * 0.0254 #meters
fuseV = (1./3.)*np.pi*(r1**2 + r1*r2 + r2**2)*h
print("Fuselage:", rounds(fuseV), "cu meters")

#Value of the heat exchanger
exchR = 0.5
exchL = 0.92
oneExchV = np.pi*exchR**2*exchL
#Double the volume for the other two exchanger
exchV = oneExchV * 2

#Volume of compressors & turbines
volComp = np.pi * 0.25**2 * 0.5
volTurb = np.pi * 0.25**2 * 0.1
volMachinery = volComp*2 + volTurb*3

emptyV = fuseV - (exchV + volMachinery)
emptyV *= 0.8
#print(np.floor(emptyV))

topFoilV = 17.192 * 0.0283168 * 0.75
botFoilV = 16.159 * 0.0283168 * 0.75
tailFoilV = 2.946 * 0.0283168 * 0.75
emptyV += topFoilV + botFoilV + tailFoilV
print("Empty Space:", rounds(emptyV),"cu meters")

Fuselage: 22.3 cu meters
Empty Space: 17.2 cu meters


In [19]:
## GETTING MASSES
turbMass = 58.4 #kg
compMass = 73.5 #kg
vehEmpMass = 5051.142 * 0.453592 #kg
exchMass = 100 #kg, very rough
totMass = exchMass *2 + turbMass * 3 + compMass *2 + vehEmpMass
totMass *= 1.10
print("Dry mass:",totMass)

#print(8192.8*0.453592)

Dry mass: 3094.6933622704005
